# Test Correspondences

In [5]:
%matplotlib widget
import numpy as np
import os
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import planeslam.io as io
from planeslam.general import NED_to_ENU
from planeslam.scan import pc_to_scan
from planeslam.registration import extract_corresponding_features, get_correspondences

os.environ['KMP_DUPLICATE_LIB_OK']='True'

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Read in airsim LiDAR and pose data

In [6]:
# Read in point cloud data
binpath = os.path.join(os.getcwd(), '..', '..', 'data', 'airsim', 'blocks_60_samples_loop_closure', 'lidar', 'Drone0')
PC_data = io.read_lidar_bin(binpath)

In [7]:
# Read in ground-truth poses (in drone local frame)
posepath = os.path.join(os.getcwd(), '..', '..', 'data', 'airsim', 'blocks_60_samples_loop_closure', 'poses', 'Drone0')
drone_positions, drone_orientations = io.read_poses(posepath)

In [8]:
# Convert to ENU
num_scans = len(PC_data)

for i in range(num_scans):
    PC_data[i] = NED_to_ENU(PC_data[i])

drone_positions = NED_to_ENU(drone_positions)
drone_orientations = NED_to_ENU(drone_orientations)

Extract planes 

In [9]:
scans = num_scans * [None]

for i in range(num_scans):
    scans[i] = pc_to_scan(PC_data[i])

basis z normal  [-2.03979337e-04 -3.31921260e-03  9.99994471e-01]
basis z normal  [ 0.27008259 -0.00251025  0.96283389]
basis z normal  [ 0.10800408 -0.00706226  0.99412537]
basis z normal  [ 0.04729154 -0.00157716  0.99887988]
basis z normal  [ 2.70742521e-02 -5.32087294e-04  9.99633284e-01]
basis z normal  [2.03750999e-02 8.13065713e-04 9.99792075e-01]
basis z normal  [0.01846569 0.00135577 0.99982858]
basis z normal  [1.91238758e-02 8.44883117e-04 9.99816765e-01]
basis z normal  [1.88252680e-02 5.77238730e-04 9.99822622e-01]
basis z normal  [1.82491266e-02 6.99921778e-04 9.99833226e-01]
basis z normal  [1.78488799e-02 6.64361488e-04 9.99840475e-01]
basis z normal  [1.74295671e-02 7.84964802e-04 9.99847785e-01]
basis z normal  [1.68955900e-02 4.27032548e-04 9.99857168e-01]
basis z normal  [ 0.01587183 -0.00253236  0.99987083]
basis z normal  [ 0.01640614 -0.00609369  0.99984684]
basis z normal  [-0.30129019  0.32655507  0.89587165]
basis z normal  [-0.29235625  0.0899313   0.95207152

In [10]:
# Plot 2 scans
source_idx = 1
target_idx = 0
source = scans[source_idx]
target = scans[target_idx]

fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'surface'}]])

for t in source.plot_trace(show_normals=True):
    fig.add_trace(t, row=1, col=1)

for t in target.plot_trace(show_normals=True):
    fig.add_trace(t, row=1, col=2)

fig.update_layout(width=1600, height=700, scene=dict(aspectmode='data'), scene2=dict(aspectmode='data'))
fig.show()

correspondences = get_correspondences(source, target)
print(correspondences)

[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9)]


In [ ]:
n = len(source.planes) # source P
m = len(target.planes) # target Q
score_mat = np.zeros((n,m))
for i in range(n):
    for j in range(m):
        n1 = source.planes[i].normal
        n2 = target.planes[j].normal
        c1 = source.planes[i].center
        c2 = target.planes[j].center
        a1 = source.planes[i].area()
        a2 = target.planes[j].area()
        score_mat[i,j] = 100 * np.linalg.norm(n1 - n2) + np.linalg.norm(c1 - c2) + 0.1 * np.abs(a1 - a2)
        #score_mat[i,j] = np.linalg.norm(c1 - c2)

In [ ]:
from scipy.optimize import linear_sum_assignment

matches = linear_sum_assignment(score_mat)
matches

In [12]:
from planeslam.registration import robust_GN_register

R_hat, t_hat = robust_GN_register(source, target)

print(R_hat)
print(t_hat)

[[ 9.62780411e-01 -1.13919939e-03 -2.70282414e-01]
 [ 8.78212591e-04  9.99999024e-01 -1.08653904e-03]
 [ 2.70283388e-01  8.08733084e-04  9.62780471e-01]]
[[-1.157906  ]
 [ 0.00935947]
 [-0.27539617]]


In [14]:
target.basis @ source.basis.T

array([[ 9.62780411e-01, -1.13919939e-03, -2.70282414e-01],
       [ 8.78212591e-04,  9.99999024e-01, -1.08653904e-03],
       [ 2.70283388e-01,  8.08733084e-04,  9.62780471e-01]])

In [ ]:
from planeslam.geometry.util import quat_to_rot_mat

R_1 = quat_to_rot_mat(drone_orientations[target_idx])
R_2 = quat_to_rot_mat(drone_orientations[source_idx])
R_gt = R_2 @ R_1.T

t_gt = drone_positions[source_idx] - drone_positions[target_idx]
print(R_gt)
print(t_gt)

In [ ]:
source.transform(R_hat, t_hat.flatten())

In [ ]:
fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'surface'}]])

for t in source.plot_trace(show_normals=True):
    fig.add_trace(t, row=1, col=1)

for t in target.plot_trace(show_normals=True):
    fig.add_trace(t, row=1, col=2)

fig.update_layout(width=1200, height=500, scene=dict(aspectmode='data'), scene2=dict(aspectmode='data'))
fig.show()

In [ ]:
fig = go.Figure(data=source.plot_trace()+target.plot_trace())
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()